In [1]:
import pandas as pd
from datetime import datetime

%matplotlib inline

In [2]:
nba2006 = pd.read_csv('./data/2006-2007_NBA_Box_Score_Team_Stats.csv')
nba2007 = pd.read_csv('./data/2007-2008_NBA_Box_Score_Team_Stats.csv')
nba2008 = pd.read_csv('./data/2008-2009_NBA_Box_Score_Team_Stats.csv')
nba2009 = pd.read_csv('./data/2009-2010_NBA_Box_Score_Team_Stats.csv')
nba2010 = pd.read_csv('./data/2010-2011_NBA_Box_Score_Team_Stats.csv')
nba2011 = pd.read_csv('./data/2011-2012_NBA_Box_Score_Team_Stats.csv')
nba2012 = pd.read_csv('./data/2012-2013_NBA_Box_Score_Team_Stats.csv')
nba2013 = pd.read_csv('./data/2013-2014_NBA_Box_Score_Team-Stats.csv')
nba2014 = pd.read_csv('./data/2014-2015_NBA_Box_Score_Team-Stats.csv')
#nba2015 = pd.read_csv('./data/2015-2016_NBA_Box_Score_Team-Stats.csv')
#nba2016 = pd.read_csv('./data/2016-2017_NBA_Box_Score_Team-Stats.csv')
#nba2017 = pd.read_csv('./data/2017-2018_NBA_Box_Score_Team-Stats.csv')

A couple of column names changed in 2014

In [3]:
nba2006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619 entries, 0 to 2618
Data columns (total 50 columns):
DATASET             2618 non-null object
DATE                2618 non-null object
TEAMS               2618 non-null object
VENUE               2618 non-null object
1Q                  2618 non-null float64
2Q                  2618 non-null float64
3Q                  2618 non-null float64
4Q                  2618 non-null float64
OT1                 180 non-null float64
OT2                 34 non-null float64
OT3                 2 non-null float64
OT4                 0 non-null float64
F                   2618 non-null float64
MIN                 2618 non-null float64
FG                  2618 non-null float64
FGA                 2618 non-null float64
3P                  2618 non-null float64
3PA                 2618 non-null float64
FT                  2618 non-null float64
FTA                 2618 non-null float64
OR                  2618 non-null float64
DR                  2618

In [4]:
nba2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2638 entries, 0 to 2637
Data columns (total 51 columns):
DATASET             2638 non-null object
DATE                2638 non-null object
TEAMS               2638 non-null object
VENUE               2638 non-null object
1Q                  2638 non-null int64
2Q                  2638 non-null int64
3Q                  2638 non-null int64
4Q                  2638 non-null int64
OT1                 176 non-null float64
OT2                 28 non-null float64
OT3                 6 non-null float64
OT4                 0 non-null float64
F                   2638 non-null int64
MIN                 2638 non-null float64
FG                  2638 non-null int64
FGA                 2638 non-null int64
3P                  2638 non-null int64
3PA                 2638 non-null int64
FT                  2638 non-null int64
FTA                 2638 non-null int64
OR                  2638 non-null int64
DR                  2638 non-null int64
TOT     

In [5]:
diff = {'OPENING SPREAD' : 'spread',
        'OPENING TOTAL'  : 'total',
        'CLOSING ODDS'   : 'closing',
        'Unnamed: 36'    : 'unnamed: 35',
        'Unnamed: 37'    : 'unnamed: 36',
        'Unnamed: 38'    : 'unnamed: 37',
        'Unnamed: 39'    : 'unnamed: 38'
       }

nba2013.rename(columns = diff, inplace = True)
nba2014.rename(columns = diff, inplace = True)

nba2013.drop(columns = ['TO TO'], inplace =True)
nba2014.drop(columns = ['TO TO'], inplace = True)

In [6]:
nba2014.head()

,DATASET,DATE,TEAMS,VENUE,1Q,2Q,3Q,4Q,OT1,OT2,...,CREW REFEREES,OPENING ODDS,spread,total,MOVEMENTS,closing,MONEYLINE,HALFTIME,BOX SCORE,ODDS
0,2014-2015 Regular Season,10/28/2014,Orlando,Road,25,16,23,20,NaN,NaN,...,David Jones,196.0,9.5,196.0,191 / 190.5 / 191.5,191.0,380,94.5,NaN,NaN
1,2014-2015 Regular Season,10/28/2014,New Orleans,Home,24,24,30,23,NaN,NaN,...,Dedric Taylor,-9.5,-9.5,196.0,-10.5 / -9.5 / -9 -05,-9.0,-475,-3.5,Boxscore,Odds
2,2014-2015 Regular Season,10/28/2014,Dallas,Road,24,29,20,27,NaN,NaN,...,Pat Fraher,206.5,6.0,206.5,203.5 / 203.5u14 / 203,203.5,140,102,NaN,NaN
3,2014-2015 Regular Season,10/28/2014,San Antonio,Home,26,19,31,25,NaN,NaN,...,Haywoode Workman,-6.0,-6.0,206.5,-4 -05 / -4 -07 / -3.5 -15,-3.5,-165,-4,Boxscore,Odds
4,2014-2015 Regular Season,10/28/2014,Houston,Road,31,31,23,23,NaN,NaN,...,Courtney Kirkland,-6.5,-6.5,209.0,-6.5 / -7 / -7 -09,-7.0,-300,+1.5 -20,NaN,NaN


In [7]:
nba2006.head(10).T

,0,1,2,3,4,5,6,7,8,9
DATASET,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season
DATE,10/31/06,10/31/06,10/31/06,10/31/06,11/01/06,11/01/06,11/01/06,11/01/06,11/01/06,11/01/06
TEAMS,Chicago,Miami,Phoenix,LA Lakers,Indiana,Charlotte,Chicago,Orlando,Atlanta,Philadelphia
VENUE,Road,Home,Road,Home,Road,Home,Road,Home,Road,Home
1Q,22,16,41,26,23,27,20,32,14,29
2Q,37,14,17,27,26,23,25,31,21,20
3Q,21,21,21,34,28,23,25,23,23,24
4Q,28,15,27,27,29,26,24,23,17,15
OT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OT2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### NBA csv File Structure

Currently, the NBA data has two records for each game one with the home team data and one with the away team data.  The unit of analysis for this study is the game so the data needs to be rearranged to use the game as the unit of analysis.  Therefore, each even row must be appended to the odd row above it to make the game the unit of analysis

In [8]:


def merge_home_away(df):
    
    
  
    #make all column names lower snake case
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    
    #drop any empty data 
    df.date.dropna(inplace = True)
    #replace missing data with zeros
    missing_list = ['ot1', 'ot2', 'ot3', 'ot4']
    
    
    #some missing moneylines going to set them to zero for now look explore later
    df['moneyline'].fillna(0, inplace = True)
    df['movements'].fillna('none', inplace = True)
    
    
    for ot in missing_list:
        df[ot].fillna(0, inplace = True)
    
    
    #split up the rows
    df_road =  df[df.index %2 == 0]
    df_home =  df[df.index %2 == 1] 
    
    #drop unnessecary columns in df1
    
    df_home.drop(columns = ['box_score', 
                            'odds',
                            'halftime',
                            'main_referee'
                           ], inplace = True)
    
    #drop redundant columns in df2
    df_road.drop(columns = ['dataset',
                        'date',
                        'venue', 
                        'pts', 
                        'poss',
                        'box_score', 
                        'odds',
                        'opening_odds', 
                        'halftime'], inplace = True)
    
    #rename columns df1 and df2 
    
    home =  {'starting_lineup' : 'home_starter1',
             'unnamed:_35'     : 'home_starter2',
             'unnamed:_36'     : 'home_starter3',
             'unnamed:_37'     : 'home_starter4',
             'unnamed:_38'     : 'home_starter5',
             'f'               : 'home_score'
            }
   
    df_home.rename(columns = home, inplace = True) 
    
    away =  {'teams'     : 'away_team',
             '1q'        : 'away_1q',
             '2q'        : 'away_2q',
             '3q'        : 'away_3q',
             '4q'        : 'away_4q',
             'ot1'       : 'away_ot1',
             'ot2'       : 'away_ot2',
             'ot3'       : 'away_ot3',
             'ot4'       : 'away_ot4',
             'f'         : 'away_score',
             'min'       : 'away_min',
             'fg'        : 'away_fg',
             'fga'       : 'away_fga',
             '3p'        : 'away_3p',
             '3pa'       : 'away_3pa',
             'ft'        : 'away_ft',
             'fta'       : 'away_fta',
             'or'        : 'away_or',
             'dr'        : 'away_dr',
             'tot'       : 'away_total_reb',
             'a'         : 'away_assists',
             'pf'        : 'away_fouls',
             'st'        : 'away_steals',
             'or'        : 'away_or',
             'to'        : 'away_turnovers',
             'bl'        : 'away_blocks',
             'poss'      : 'away_poss',
             'pace'      : 'away_pace',
             'oeff'      : 'away_off_eff',
             'deff'      : 'away_def_eff',
             'rest_days' : 'away_rest',
             'starting_lineup' : 'away_starter1',
             'unnamed:_35'     : 'away_starter2',
             'unnamed:_36'     : 'away_starter3',
             'unnamed:_37'     : 'away_starter4',
             'unnamed:_38'     : 'away_starter5',
             'crew_referees'   : 'ref_3',
             'spread'          : 'away_line',
             'total'           : 'away_total',
             'movements'       : 'total_moves',
             'closing'         : 'closing total',
             'moneyline'       : 'away_moneyline',
             }
    df_road.rename(columns = away, inplace = True)
    

    
    #reset the indexs to merge the files
    df_home.reset_index(inplace = True)
    df_road.reset_index(inplace = True)
                         
    #merge data so game becomes rather than team unit of analysis                      
    new =  pd.concat([df_home,df_road], axis = 1)
  
    #establish data as a date/time variable
    new['date'] = pd.to_datetime(new['date'])
    
    #add cover information
    new['line_cv'] = new.home_score - new.away_score + new.spread
    
    new['cover'] = new['line_cv'].map(lambda x: 1 if x >0 else 0)
    
    #add win and score difference info
    
    
    new['home_win_margin'] = (new['home_score'] - new['away_score'])
    new['win'] = new['home_win_margin'].map(lambda x: 1 if x > 0 else 0)
    new['away_win_margin'] = -new['home_win_margin']
    new['away_win'] = new['win'].map(lambda x: 0 if x == 1 else 1)
    
    
    
    #call second function
    new2 = create_home_mov_ave(new)
    new3 = create_away_mov_ave(new2)
    return new3

### Calulating my dependent variable.  

Line_cv is the amount the game went over or under the spread, and then cover is 1 for a home team spread victory and 0 for an away team spread victory.  This leave us with to possible methods of prediction.  We can use regression to predict the cover amount or we can use categorization to predict which category it fall into.  We also could try and predict individual team scores using regression and use those as a prediction

In [9]:
def create_home_mov_ave(df):
    

    
    #add underscores to teams
    df['teams'] = [str(team).replace(' ', '_') for team in df['teams']]
    
    #loop through teams
    teams = ['Atlanta','Boston','Charlotte','Chicago',
             'Cleveland','Dallas','Denver','Detroit',
             'Golden_State','Houston','Indiana','LA_Clippers',
             'LA_Lakers','Memphis','Miami','Milwaukee',
             'Minnesota','New_Jersey','New_Orleans','New_York',
             'Orlando','Philadelphia','Phoenix','Portland',
             'Sacramento','San_Antonio','Seattle','Toronto',
             'Utah','Washington']
    

    columns = ['1q', '2q', '3q', '4q',
               'ot1', 'ot2', 'ot3', 'ot4', 'home_score', 'min', 'fg', 
               'fga', '3p', '3pa', 'ft','fta', 'or', 'dr', 'tot', 
               'a', 'pf', 'st', 'to', 'pts', 'poss','home_score',
               'pace', 'oeff', 'deff', 'bl', 'win', 'home_win_margin',
               'away_score']
    #create empty data frame to put in results
    home_vars = pd.DataFrame()
    
    for team in teams:
        #subset by team and sort by date
        df_team = df[df['teams'] == team]
        df.sort_values('date', inplace = True)
      
        #create 5 game moving averages
        for column in columns:
            column_new = 'mov_5_' + column
            df_team[column_new] = df_team[column].rolling(5).mean().shift(1)
        
            
        
        
        #create average home scores and win totals
        home_win_pcts = []
        home_win_pct = 0
        
        #create average home win margin scores
        home_ave_win_margins = []
        home_ave_win_margin = 0
        
        for i in range(len(df_team['win'])):
            
            #cumulative win percentages
            home_win_pct += df_team.iloc[i, 92]
            home_win_pcts.append(home_win_pct/(i+1))
            
            #average home win margin
            home_ave_win_margin += df_team.iloc[i,91 ]
            home_ave_win_margins.append(home_ave_win_margin/(i+1))
       
        #assign the home win percents
        df_team['home_win_pct'] = home_win_pcts    
        df_team['home_win_pct'] = df_team.home_win_pct.shift(1)
        
        #Assign average winning margin
        df_team['home_ave_win_margin'] = home_ave_win_margins
        df_team['home_ave_win_margin'] = df_team.home_ave_win_margin.shift(1)
        
        home_vars = home_vars.append(df_team)
    return home_vars 



In [10]:
def create_away_mov_ave(df):
    

    
    #add underscores to teams
    df['away_team'] = [str(team).replace(' ', '_') for team in df['away_team']]
    
    #loop through teams
    teams = ['Atlanta','Boston','Charlotte','Chicago',
             'Cleveland','Dallas','Denver','Detroit',
             'Golden_State','Houston','Indiana','LA_Clippers',
             'LA_Lakers','Memphis','Miami','Milwaukee',
             'Minnesota','New_Jersey','New_Orleans','New_York',
             'Orlando','Philadelphia','Phoenix','Portland',
             'Sacramento','San_Antonio','Seattle','Toronto',
             'Utah','Washington']
    

    columns = ['away_1q', 'away_2q', 'away_3q', 'away_4q',
               'away_ot1', 'away_ot2', 'away_ot3', 'away_ot4', 
               'away_score', 'away_min', 'away_fg', 'away_fga', 
               'away_3p', 'away_3pa', 'away_ft','away_fta',
               'away_or', 'away_dr', 'away_total_reb', 
               'away_assists', 'away_fouls', 'away_steals', 
               'away_turnovers','away_blocks', 
               'away_off_eff',
               'away_def_eff', 'away_win', 'away_win_margin']
    #create empty data frame to put in results
    away_vars = pd.DataFrame()
    
    for team in teams:
        
        df_team = df[df['away_team'] == team]
        df.sort_values('date', inplace = True)
        for column in columns:
            column_new = 'mov_5_' + column
            df_team[column_new] = df_team[column].rolling(5).mean().shift(1)
        
        
        #create average away win totals
        away_win_pcts = []
        away_win_pct = 0
        
        #create average home win margin scores
        away_ave_win_margins = []
        away_ave_win_margin = 0
        
        for i in range(len(df_team['win'])):
            
            #cumulative win percentages
            away_win_pct += df_team.iloc[i, 94]
            away_win_pcts.append(away_win_pct/(i+1))
            
            #average home win margin
            away_ave_win_margin += df_team.iloc[i,93 ]
            away_ave_win_margins.append(away_ave_win_margin/(i+1))
       
        #assign the home win percents
        df_team['away_win_pct'] = away_win_pcts    
        df_team['away_win_pct'] = df_team.away_win_pct.shift(1)
        
        #Assign average winning margin
        df_team['away_ave_win_margin'] = away_ave_win_margins
        df_team['away_ave_win_margin'] = df_team.away_ave_win_margin.shift(1)
        
        
        away_vars = away_vars.append(df_team)
    
        
    return away_vars



In [11]:
nba2006_trans = merge_home_away(nba2006)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipyker

In [12]:
nba2006_trans.T

,622,960,590,1058,615,791,51,1187,844,268,...,1033,1045,1089,1098,1135,1164,1177,1222,1234,1244
index,1245,1921,1181,2117,1231,1583,103,2375,1689,537,...,2067,2091,2179,2197,2271,2329,2355,2445,2469,2489
dataset,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,...,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2007 Playoffs,2007 Playoffs
date,2007-01-24 00:00:00,2007-03-14 00:00:00,2007-01-20 00:00:00,2007-03-28 00:00:00,2007-01-23 00:00:00,2007-02-20 00:00:00,2006-11-07 00:00:00,2007-04-14 00:00:00,2007-02-26 00:00:00,2006-12-06 00:00:00,...,2007-03-24 00:00:00,2007-03-26 00:00:00,2007-04-01 00:00:00,2007-04-03 00:00:00,2007-04-07 00:00:00,2007-04-11 00:00:00,2007-04-13 00:00:00,2007-04-18 00:00:00,2007-04-22 00:00:00,2007-04-25 00:00:00
teams,Boston,Boston,Charlotte,Charlotte,Chicago,Chicago,Cleveland,Cleveland,Dallas,Denver,...,LA_Clippers,Utah,Milwaukee,Charlotte,New_Jersey,Miami,Atlanta,Indiana,Cleveland,Cleveland
venue,Home,Home,Home,Home,Home,Home,Home,Home,Home,Home,...,Home,Home,Home,Home,Home,Home,Home,Home,Home,Home
1q,28,26,25,25,27,26,20,28,27,24,...,21,13,35,34,22,25,20,18,27,23
2q,16,24,23,24,14,22,18,28,33,20,...,26,22,16,29,30,10,15,35,21,28
3q,18,36,27,26,29,28,33,29,27,31,...,32,35,34,34,28,27,26,20,26,24
4q,14,23,29,26,24,30,19,25,23,21,...,32,33,22,25,24,23,24,22,23,34
ot1,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,16,0,0,0,0,0


In [13]:
def merge_years(df1,df2):
    return pd.concat([df1,df2], axis = 0)
        

In [14]:
nba2006_trans.home_win_margin.value_counts()

 5.0     49
 6.0     49
 9.0     47
 8.0     44
 2.0     43
-2.0     42
-3.0     42
-4.0     42
 4.0     42
 10.0    42
 13.0    40
-7.0     39
 3.0     39
 7.0     37
-8.0     37
 15.0    33
-6.0     33
-1.0     33
-5.0     32
 11.0    32
 12.0    32
-9.0     31
 1.0     29
-11.0    25
 18.0    22
 16.0    20
 19.0    19
-12.0    19
-14.0    18
-10.0    18
         ..
 27.0     9
 28.0     6
 29.0     6
-19.0     5
 32.0     5
-21.0     5
-24.0     5
 30.0     4
-26.0     4
-23.0     4
 31.0     3
-25.0     3
-20.0     3
-30.0     2
-31.0     2
 37.0     2
 33.0     2
 34.0     1
-37.0     1
-41.0     1
-42.0     1
-33.0     1
-36.0     1
 40.0     1
-50.0     1
 45.0     1
-28.0     1
-35.0     1
-27.0     1
 39.0     1
Name: home_win_margin, Length: 75, dtype: int64

In [15]:

nba2007_trans = merge_home_away(nba2007)
nba2008_trans = merge_home_away(nba2008)
nba2009_trans = merge_home_away(nba2009)
nba2010_trans = merge_home_away(nba2010)
nba2011_trans = merge_home_away(nba2011)
nba2012_trans = merge_home_away(nba2012)
nba2013_trans = merge_home_away(nba2013)
nba2014_trans = merge_home_away(nba2014)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipyker

In [16]:
nba2013_trans.drop(columns = ['to_to'])
nba2014_trans.drop(columns = ['to_to']) 


,index,dataset,date,teams,venue,1q,2q,3q,4q,ot1,...,mov_5_away_fouls,mov_5_away_steals,mov_5_away_turnovers,mov_5_away_blocks,mov_5_away_off_eff,mov_5_away_def_eff,mov_5_away_win,mov_5_away_win_margin,away_win_pct,away_ave_win_margin
579,1159,2014-2015 Regular Season,2015-01-14,Boston,Home,24,21,29,17,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
786,1573,2014-2015 Regular Season,2015-02-11,Boston,Home,13,21,27,28,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,14.000000
70,141,2014-2015 Regular Season,2014-11-07,Charlotte,Home,22,25,23,27,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,6.500000
1085,2171,2014-2015 Regular Season,2015-03-28,Charlotte,Home,27,30,33,25,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,3.333333
601,1203,2014-2015 Regular Season,2015-01-17,Chicago,Home,16,23,28,32,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250000,-1.250000
1217,2435,2014-2015 Regular Season,2015-04-15,Chicago,Home,17,25,28,21,0.0,...,19.6,8.6,12.4,5.2,108.18,107.32,0.4,0.6,0.400000,0.600000
132,265,2014-2015 Regular Season,2014-11-15,Cleveland,Home,41,30,30,26,0.0,...,21.0,8.0,13.0,5.8,101.66,105.14,0.2,-3.4,0.333333,-0.500000
369,739,2014-2015 Regular Season,2014-12-17,Cleveland,Home,34,26,15,23,0.0,...,20.0,8.0,14.2,6.4,101.78,111.74,0.2,-9.8,0.285714,-5.142857
1301,2603,2015 Playoffs,2015-05-24,Cleveland,Home,21,27,33,23,10.0,...,16.8,7.0,12.2,6.8,110.26,113.02,0.4,-3.4,0.375000,-0.875000
1303,2607,2015 Playoffs,2015-05-26,Cleveland,Home,32,27,26,33,0.0,...,18.4,7.6,11.6,6.6,110.64,110.72,0.4,-1.0,0.333333,-1.111111


In [17]:
nba2013_trans.head().T


,458,851,97,993,490
index,917,1703,195,1987,981
dataset,2013-2014 Regular Season,2013-2014 Regular Season,2013-2014 Regular Season,2013-2014 Regular Season,2013-2014 Regular Season
date,2013-12-31 00:00:00,2014-02-26 00:00:00,2013-11-11 00:00:00,2014-03-17 00:00:00,2014-01-04 00:00:00
teams,Boston,Boston,Charlotte,Charlotte,Chicago
venue,Home,Home,Home,Home,Home
1q,23,25,29,28,26
2q,21,33,25,24,24
3q,23,23,16,16,17
4q,24,34,24,15,24
ot1,0,0,0,0,0


In [18]:
nba_combined = merge_years(nba2006_trans,nba2007_trans)


In [19]:
nba_combined = merge_years(nba_combined,nba2008_trans)
nba_combined = merge_years(nba_combined,nba2009_trans)
nba_combined = merge_years(nba_combined,nba2010_trans)
nba_combined = merge_years(nba_combined,nba2011_trans)
nba_combined = merge_years(nba_combined,nba2012_trans)


In [20]:
nba_combined.columns[0:50]

Index(['index', 'dataset', 'date', 'teams', 'venue', '1q', '2q', '3q', '4q',
       'ot1', 'ot2', 'ot3', 'ot4', 'home_score', 'min', 'fg', 'fga', '3p',
       '3pa', 'ft', 'fta', 'or', 'dr', 'tot', 'a', 'pf', 'st', 'to', 'bl',
       'pts', 'poss', 'pace', 'oeff', 'deff', 'rest_days', 'starting_lineups',
       'home_starter2', 'home_starter3', 'home_starter4', 'home_starter5',
       'crew_referees', 'opening_odds', 'spread', 'total', 'movements',
       'closing', 'moneyline', 'index', 'away_team', 'away_1q'],
      dtype='object')

In [21]:
nba2013_trans.columns[0:50]

Index(['index', 'dataset', 'date', 'teams', 'venue', '1q', '2q', '3q', '4q',
       'ot1', 'ot2', 'ot3', 'ot4', 'home_score', 'min', 'fg', 'fga', '3p',
       '3pa', 'ft', 'fta', 'or', 'dr', 'tot', 'a', 'pf', 'st', 'to', 'bl',
       'pts', 'poss', 'pace', 'oeff', 'deff', 'rest_days', 'starting_lineups',
       'home_starter2', 'home_starter3', 'home_starter4', 'home_starter5',
       'crew_referees', 'opening_odds', 'spread', 'total', 'movements',
       'closing', 'moneyline', 'index', 'away_team', 'away_1q'],
      dtype='object')

In [22]:
nba_combined = merge_years(nba_combined,nba2013_trans)
nba_combined = merge_years(nba_combined,nba2014_trans)

### Transforming categorical variables 

Even in decision tree classifiers we need to have float data.  So here I transform a list of categorical variables into a list of integer variables for the decision tree analysis.

In [23]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
nba_combined['home_rest'] = lbl.fit_transform(list(nba_combined['rest_days'].values))
nba_combined['away_rest'] = lbl.fit_transform(list(nba_combined['away_rest'].values)) 
nba_combined['main_referee'] = lbl.fit_transform(list(nba_combined['main_referee'].values))   
nba_combined['ref_3'] = lbl.fit_transform(list(nba_combined['ref_3'].values))    
nba_combined['ref_2'] = lbl.fit_transform(list(nba_combined['crew_referees'].values)) 

### Dropping columns that are no longer necessary.  

The NBA data that relates to the particular game that has just been played is not longer necessary so we can drop those columns to make sure that they are not used in the analysis as we do not have access to that data.   

In [24]:
nba_combined.drop(columns = ['away_1q', 'away_2q', 'away_3q', 'away_4q',
               'away_ot1', 'away_ot2', 'away_ot3', 'away_ot4', 
               'away_min', 'away_fg', 'away_fga', 
               'away_3p', 'away_3pa', 'away_ft','away_fta',
               'away_or', 'away_dr', 'away_total_reb', 
               'away_assists', 'away_fouls', 'away_steals', 
               'away_turnovers','away_blocks', 
               'away_off_eff','away_def_eff', '1q', '2q', '3q', '4q',
               'ot1', 'ot2', 'ot3', 'ot4', 'min', 'fg', 
               'fga', '3p', '3pa', 'ft','fta', 'or', 'dr', 'tot', 
               'a', 'pf', 'st', 'to', 'pts', 'poss',
               'pace', 'oeff', 'deff', 'bl', 'rest_days',
               'crew_referee'    
                            ], inplace = True)

In [25]:
nba_combined.to_csv('./data/nba_combined.csv', index = False)